In [1]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport GLS_Apr_weekday_PM

No dicts found; please check load_dicts...
Optimize a model with 314 rows, 314 columns and 314 nonzeros
Model has 49455 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [3e-03, 3e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 314 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 314 columns, 0 nonzeros
Presolved model has 49455 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.58419864e+07  0.00000000e+00  2.72e+04 2.61e+02  9.25e+05     0s
   1   1.15707595e+07 -1.55939797e+07  3.62e+03 2.27e-13  2.15e+05     0s
   2   4.24056896e+06 -5.41296606e+06  3.62e-03 2.27e-13  3.07e+04     0s
   3   4.40

In [2]:
xi_list = GLS_Apr_weekday_PM.xi_list
P = GLS_Apr_weekday_PM.P
L = GLS_Apr_weekday_PM.L  # dimension of xi

314

In [3]:
size(P, 1), size(P, 2)

(56,314)

In [4]:
P

56×314 Array{Float64,2}:
 1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [5]:
using JuMP

mGLSJulia = Model()

@variable(mGLSJulia, lam[1:size(P,1)] >= 0)

@variable(mGLSJulia, p[1:size(P,1), 1:size(P,2)] >= 0)

for i = 1:size(P,1)
    for j = 1:size(P,2)
        if P[i,j] == 0
            @constraint(mGLSJulia, p[i,j] == 0)
        end
    end
end
            
for i = 1:size(P,1)
    @NLconstraint(mGLSJulia, sum{p[i,j], j = 1:size(P,2)} == 1)
end

for l = 1:L
    @NLconstraint(mGLSJulia, sum{p[i,l] * lam[i], i = 1:size(P,1)} == xi_list[l])
end
    
@NLobjective(mGLSJulia, Min, sum{p[1,j], j = 1:size(P,2)})  # play no actual role, but could not use zero objective

solve(mGLSJulia)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    70022
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:    52752

Total number of variables............................:    17640
                     variables with only lower bounds:    17640
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

:Optimal

In [6]:
getvalue(lam)

56-element Array{Float64,1}:
  364.031
  325.364
  387.088
  543.744
  531.093
  533.457
  187.046
  511.458
  326.3  
  459.388
  581.073
  663.681
  610.212
    ⋮    
  714.337
  517.884
  849.503
 1621.38 
  188.789
  239.612
  234.682
  514.833
  383.71 
  402.015
  566.538
  282.776

In [7]:
getobjectivevalue(mGLSJulia)

1.0000000000000075

In [8]:
GLS_Apr_weekday_PM.saveDemandVec(getvalue(lam))

In [9]:
getvalue(p)

56×314 Array{Float64,2}:
 0.685339     0.171459     0.143202     …  0.0          0.0        
 8.54994e-16  8.54969e-16  8.54967e-16     0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 0.0          0.0          0.0          …  0.0          0.0        
 1.98009e-15  1.98008e-15  1.98008e-15     1.14689e-15  1.14689e-15
 0.0          0.0          0.0             0.0          0.0        
 8.40237e-16  8.40211e-16  8.4021e-16      0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 0.0          0.0          0.0          …  0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 ⋮                                      ⋱                          
 0.0          0.0      

In [10]:
route_choice_matrix_Apr_PM = getvalue(p)

56×314 Array{Float64,2}:
 0.685339     0.171459     0.143202     …  0.0          0.0        
 8.54994e-16  8.54969e-16  8.54967e-16     0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 0.0          0.0          0.0          …  0.0          0.0        
 1.98009e-15  1.98008e-15  1.98008e-15     1.14689e-15  1.14689e-15
 0.0          0.0          0.0             0.0          0.0        
 8.40237e-16  8.40211e-16  8.4021e-16      0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 0.0          0.0          0.0          …  0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 ⋮                                      ⋱                          
 0.0          0.0      

In [11]:
sum(route_choice_matrix_Apr_PM[1, :])

1.0000000000001918

In [12]:
outfile = open("../temp_files/route_choice_CDC_Apr_PM.json", "w")

JSON.print(outfile, route_choice_matrix_Apr_PM)

close(outfile)

# Route flow analysis (see [Eq. (9) in M. Hazelton (2000); Page 556])

$\textbf{P}'\textbf{g}$ equals the route flow vector. It seems hard to uniquely determine a route flow vector corresponding to a social link flow vector.